## Imports

In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import random

## Set seed for reproducibility

In [2]:
def set_seed(seed):
    """
    Set the seed for reproducibility.

    Args:
        seed (int): Seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed to be 42
set_seed(42)

## Resnet-34

## 5-fold Cross Validation

In [3]:
# Hyperparameters
best_hyperparams = {'lr': 0.01, 'batch_size': 16}

In [4]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score
import numpy as np

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create the ResNet34 model and load pretrained weights, excluding the final fc layer
def create_resnet_model(num_classes, pretrained_path=None):
    model = models.resnet34(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)  # Replace the final layer for the new task
    
    if pretrained_path:
        # Load the pretrained model, but exclude the final fully connected layer weights
        state_dict = torch.load(pretrained_path)
        # Remove the fc layer weights from the pretrained model
        del state_dict['fc.weight']
        del state_dict['fc.bias']
        # Load the remaining layers
        model.load_state_dict(state_dict, strict=False)  # strict=False ignores missing keys

    return model.to(device)

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43636918, 0.38563913, 0.34477144],
                         std=[0.29639485, 0.2698132, 0.26158142])
])

# Set the dataset folder
dataset_folder = '../data/custom/'

# Load datasets
train_dataset = datasets.ImageFolder(root=dataset_folder + 'train', transform=transform)
test_dataset = datasets.ImageFolder(root=dataset_folder + 'test', transform=transform)

# Get the number of classes
num_classes = len(train_dataset.classes)

# Create DataLoader for train and test datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Path to the pretrained model (.pth file)
pretrained_model_path = './resnet_34_pretrained.pth'  # Replace with the correct path

# Initialize model, loss function, and optimizer
model = create_resnet_model(num_classes, pretrained_path=pretrained_model_path)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can modify the learning rate if necessary

# Test loop
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct_preds = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # Disable gradient calculation for faster inference
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            
            preds = outputs.argmax(1)
            correct_preds += (preds == labels).sum().item()
            
            # Store predictions and labels for further analysis
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # Calculate test loss and accuracy
    test_loss /= len(test_loader.dataset)
    test_acc = correct_preds / len(test_loader.dataset)
    
    # Calculate F1 score
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}, F1 Score: {f1:.4f}')


# Training loop for 20 epochs
num_epochs = 20
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_preds = 0
    
    # Loop over batches of data
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        correct_preds += (outputs.argmax(1) == labels).sum().item()
    
    # Calculate training loss and accuracy
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct_preds / len(train_loader.dataset)
    
    # Print results for this epoch
    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
    print(f'Testing Model...')
    evaluate_model(model, test_loader)

# Save the trained model
torch.save(model.state_dict(), 'ryan_bush_resnet34.pth')

print("Training completed and model saved as 'trained_model.pth'.")    
# Evaluate the model on the test set
evaluate_model(model, test_loader)


FileNotFoundError: [Errno 2] No such file or directory: '../data/custom/train'

In [5]:
import os
import matplotlib.pyplot as plt
import torchvision.utils as vutils

# Directory to save misclassified images
misclassified_dir = 'misclassified_images/'
if not os.path.exists(misclassified_dir):
    os.makedirs(misclassified_dir)

# Function to save and display misclassified images
def save_and_display_misclassified_images(misclassified_data):
    print(f"Saving {len(misclassified_data)} misclassified images to '{misclassified_dir}'")
    for i, (image, true_label, pred_label) in enumerate(misclassified_data):
        # Save misclassified image
        image_path = os.path.join(misclassified_dir, f"image_{i}_true_{true_label}_pred_{pred_label}.png")
        vutils.save_image(image, image_path)
        print(f"Saved: {image_path}")
    
    # Optionally display a few misclassified images
    num_display = min(5, len(misclassified_data))  # Display at most 5 images
    if num_display > 0:
        fig, axes = plt.subplots(1, num_display, figsize=(15, 5))
        for i in range(num_display):
            image, true_label, pred_label = misclassified_data[i]
            image = image.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C) for display
            image = image.cpu().numpy()
            image = (image * 0.26158142 + 0.34477144)  # Un-normalize based on dataset stats

            axes[i].imshow(image)
            axes[i].axis('off')
            axes[i].set_title(f"True: {true_label}, Pred: {pred_label}")
        plt.show()

# Test loop to identify misclassified images
def evaluate_and_collect_misclassified(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    misclassified_data = []  # To store misclassified images
    correct_preds = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)
            correct_preds += (preds == labels).sum().item()

            # Find misclassified images
            misclassified_idx = (preds != labels).nonzero(as_tuple=True)[0]
            for idx in misclassified_idx:
                misclassified_data.append((images[idx].cpu(), labels[idx].item(), preds[idx].item()))

            # Collect predictions and labels for metrics
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    # Calculate test accuracy
    test_acc = correct_preds / len(test_loader.dataset)
    print(f"Test Accuracy: {test_acc:.4f}")

    # Save and display misclassified images
    save_and_display_misclassified_images(misclassified_data)

# Run the evaluation and collect misclassified images
evaluate_and_collect_misclassified(model, test_loader)

Test Accuracy: 1.0000
Saving 0 misclassified images to 'misclassified_images/'
